# Lab 3: Contextual Bandit-Based News Article Recommendation

**`Course`:** Reinforcement Learning Fundamentals  
**`Student Name`:**  
**`Roll Number`:**  
**`GitHub Branch`:** firstname_U20230xxx  

# Imports and Setup

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import advanced classifiers
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

from rlcmab_sampler import sampler


# Load Datasets

In [9]:
# Load datasets
news_df = pd.read_csv("data/news_articles.csv")
train_users = pd.read_csv("data/train_users.csv")
test_users = pd.read_csv("data/test_users.csv")

print("News Articles Dataset:")
print(news_df.head())
print(f"\nShape: {news_df.shape}")

print("\n" + "="*60)
print("Train Users Dataset:")
print(train_users.head())
print(f"\nShape: {train_users.shape}")


News Articles Dataset:
                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  

## Data Preprocessing

In this section:
- Handle missing values
- Encode categorical features
- Prepare data for user classification

In [10]:
# Check for missing values in all datasets
print("=" * 60)
print("Missing Values Analysis")
print("=" * 60)
print("\nNews Articles Dataset:")
print(news_df.isnull().sum())
print(f"\nShape: {news_df.shape}")

print("\nTrain Users Dataset:")
print(train_users.isnull().sum())
print(f"\nShape: {train_users.shape}")

print("\nTest Users Dataset:")
print(test_users.isnull().sum())
print(f"\nShape: {test_users.shape}")

Missing Values Analysis

News Articles Dataset:
link                     0
headline                 6
category                 0
short_description    19712
authors              37418
date                     0
dtype: int64

Shape: (209527, 6)

Train Users Dataset:
user_id                          0
age                            698
income                           0
clicks                           0
purchase_amount                  0
session_duration                 0
content_variety                  0
engagement_score                 0
num_transactions                 0
avg_monthly_spend                0
avg_cart_value                   0
browsing_depth                   0
revisit_rate                     0
scroll_activity                  0
time_on_site                     0
interaction_count                0
preferred_price_range            0
discount_usage_rate              0
wishlist_size                    0
product_views                    0
repeat_purchase_gap (days)       0


In [11]:
# Handle missing values
# For numerical columns: fill with median
# For categorical columns: fill with mode

def handle_missing_values(df):
    """
    Handle missing values in the dataset.
    - Numerical columns: filled with median
    - Categorical columns: filled with mode
    """
    df_cleaned = df.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].isnull().sum() > 0:
            if df_cleaned[col].dtype in ['int64', 'float64']:
                # Fill numerical columns with median
                df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
            else:
                # Fill categorical columns with mode
                df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)
    
    return df_cleaned

# Clean all datasets
news_df_clean = handle_missing_values(news_df)
train_users_clean = handle_missing_values(train_users)
test_users_clean = handle_missing_values(test_users)

print("Missing values after cleaning:")
print(f"News Articles: {news_df_clean.isnull().sum().sum()}")
print(f"Train Users: {train_users_clean.isnull().sum().sum()}")
print(f"Test Users: {test_users_clean.isnull().sum().sum()}")

Missing values after cleaning:
News Articles: 57136
Train Users: 698
Test Users: 679


In [12]:
# Display dataset information
print("=" * 60)
print("Dataset Information")
print("=" * 60)

print("\nNews Articles Categories:")
print(news_df_clean['category'].value_counts())

print("\nUser Categories (Train):")
print(train_users_clean['label'].value_counts())

print("\nColumn names (Train Users):")
print(train_users_clean.columns.tolist())

Dataset Information

News Articles Categories:
category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO V

In [13]:
# Feature encoding for user data
# Separate features and labels

# For training users
X_train_full = train_users_clean.drop('label', axis=1)
y_train_full = train_users_clean['label']

# For test users
X_test = test_users_clean.drop('label', axis=1) if 'label' in test_users_clean.columns else test_users_clean

# Identify categorical columns (non-numeric)
categorical_cols = X_train_full.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train_full.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

Categorical columns: ['user_id', 'browser_version', 'region_code']
Numerical columns: ['age', 'income', 'clicks', 'purchase_amount', 'session_duration', 'content_variety', 'engagement_score', 'num_transactions', 'avg_monthly_spend', 'avg_cart_value', 'browsing_depth', 'revisit_rate', 'scroll_activity', 'time_on_site', 'interaction_count', 'preferred_price_range', 'discount_usage_rate', 'wishlist_size', 'product_views', 'repeat_purchase_gap (days)', 'churn_risk_score', 'loyalty_index', 'screen_brightness', 'battery_percentage', 'cart_abandonment_count', 'background_app_count', 'session_inactivity_duration', 'network_jitter']


In [14]:
# Encode categorical features using LabelEncoder
label_encoders = {}

X_train_encoded = X_train_full.copy()
X_test_encoded = X_test.copy()

for col in categorical_cols:
    le = LabelEncoder()
    
    # Fit on training data
    X_train_encoded[col] = le.fit_transform(X_train_full[col].astype(str))
    
    # Transform test data (handle unseen labels)
    if col in X_test_encoded.columns:
        # Create a mapping for unseen categories
        test_col_values = X_test[col].astype(str)
        X_test_encoded[col] = test_col_values.map(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1
        )
    
    label_encoders[col] = le

print("\nEncoding completed!")
print(f"Encoded training data shape: {X_train_encoded.shape}")
print(f"Encoded test data shape: {X_test_encoded.shape}")


Encoding completed!
Encoded training data shape: (2000, 32)
Encoded test data shape: (2000, 32)


In [15]:
# Display sample of encoded data
print("Sample of encoded training data:")
print(X_train_encoded.head())
print("\nData types:")
print(X_train_encoded.dtypes)

Sample of encoded training data:
   user_id   age  income  clicks  purchase_amount  session_duration  \
0     1340   NaN   23053      10           500.00             17.34   
1      491  56.0   20239      11           913.33             22.22   
2      446   NaN   13907       9          1252.62             41.57   
3     1354   NaN   26615      12           500.00             30.17   
4     1082  32.0   27958      13           500.00             65.27   

   content_variety  engagement_score  num_transactions  avg_monthly_spend  \
0          0.36661          37.29781                 3             187.44   
1          0.61370          59.36342                 5             145.15   
2          0.80368          76.78706                 7             282.03   
3          0.26499          30.19441                10             195.35   
4          0.36385          37.12153                 5             439.68   

   ...  loyalty_index  screen_brightness  battery_percentage  \
0  ...       

In [16]:
# Map news categories to standard format (Entertainment, Education, Tech, Crime)
# Based on the assignment requirements
print("=" * 60)
print("News Category Mapping")
print("=" * 60)

# The actual categories in the dataset might differ from the required ones
# We'll map them appropriately
category_mapping = {
    'U.S. NEWS': 'Crime',
    'COMEDY': 'Entertainment',
    'PARENTING': 'Education',
    'POLITICS': 'Crime',
    'WELLNESS': 'Education',
    'ENTERTAINMENT': 'Entertainment',
    'TECH': 'Tech',
    'SCIENCE': 'Tech',
    'BUSINESS': 'Tech',
    'EDUCATION': 'Education',
    'CRIME': 'Crime'
}

# Apply mapping if categories exist in news dataset
if 'category' in news_df_clean.columns:
    # Check unique categories
    print("\nOriginal categories in news dataset:")
    print(news_df_clean['category'].unique())
    
    # Map to standard categories
    news_df_clean['category_mapped'] = news_df_clean['category'].map(
        lambda x: category_mapping.get(x.upper() if isinstance(x, str) else x, x)
    )
    
    print("\nMapped categories:")
    print(news_df_clean['category_mapped'].value_counts())
    
    # Use mapped categories
    news_df_clean['category'] = news_df_clean['category_mapped']

# Define the arm mapping according to assignment specifications
print("\n" + "="*60)
print("Arm Index Mapping (j values)")
print("="*60)
arm_mapping = {
    0: ('Entertainment', 'user_1'),
    1: ('Education', 'user_1'),
    2: ('Tech', 'user_1'),
    3: ('Crime', 'user_1'),
    4: ('Entertainment', 'user_2'),
    5: ('Education', 'user_2'),
    6: ('Tech', 'user_2'),
    7: ('Crime', 'user_2'),
    8: ('Entertainment', 'user_3'),
    9: ('Education', 'user_3'),
    10: ('Tech', 'user_3'),
    11: ('Crime', 'user_3')
}

for arm_idx, (category, user) in arm_mapping.items():
    print(f"Arm {arm_idx:2d}: ({category:13s}, {user})")

# Create reverse mapping for easy lookup
category_user_to_arm = {v: k for k, v in arm_mapping.items()}
print("\nArm mapping created successfully!")

News Category Mapping

Original categories in news dataset:
<StringArray>
[     'U.S. NEWS',         'COMEDY',      'PARENTING',     'WORLD NEWS',
 'CULTURE & ARTS',           'TECH',         'SPORTS',  'ENTERTAINMENT',
       'POLITICS',     'WEIRD NEWS',    'ENVIRONMENT',      'EDUCATION',
          'CRIME',        'SCIENCE',       'WELLNESS',       'BUSINESS',
 'STYLE & BEAUTY',   'FOOD & DRINK',          'MEDIA',   'QUEER VOICES',
  'HOME & LIVING',          'WOMEN',   'BLACK VOICES',         'TRAVEL',
          'MONEY',       'RELIGION',  'LATINO VOICES',         'IMPACT',
       'WEDDINGS',        'COLLEGE',        'PARENTS', 'ARTS & CULTURE',
          'STYLE',          'GREEN',          'TASTE', 'HEALTHY LIVING',
  'THE WORLDPOST',      'GOOD NEWS',      'WORLDPOST',          'FIFTY',
           'ARTS',        'DIVORCE']
Length: 42, dtype: str

Mapped categories:
category_mapped
Crime             40541
Education         27750
Entertainment     22762
Tech              10302
TRAV

## User Classification

Train a classifier to predict the user category (`User1`, `User2`, `User3`),
which serves as the **context** for the contextual bandit.


In [17]:
# Split the training data into train and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_encoded, 
    y_train_full, 
    test_size=0.2, 
    random_state=42,
    stratify=y_train_full
)

print("Data Split Summary:")
print("=" * 60)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")
print(f"Test set size: {X_test_encoded.shape[0]} samples")
print("\nClass distribution in training set:")
print(y_train.value_counts())
print("\nClass distribution in validation set:")
print(y_val.value_counts())

Data Split Summary:
Training set size: 1600 samples
Validation set size: 400 samples
Test set size: 2000 samples

Class distribution in training set:
label
user_2    570
user_1    565
user_3    465
Name: count, dtype: int64

Class distribution in validation set:
label
user_2    142
user_1    142
user_3    116
Name: count, dtype: int64


In [18]:
# Note: We will use state-of-the-art gradient boosting classifiers:
# - LightGBM: Fast, efficient, and excellent for structured data
# - XGBoost: Robust and widely used in competitions
# - CatBoost: Handles categorical features natively
print("Using advanced gradient boosting classifiers...")

Using advanced gradient boosting classifiers...


In [19]:
# LightGBM Classifier
print("=" * 60)
print("Training LightGBM Classifier")
print("=" * 60)

lgb_classifier = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

lgb_classifier.fit(X_train, y_train)

# Predictions
y_pred_lgb = lgb_classifier.predict(X_val)

# Evaluation
lgb_accuracy = accuracy_score(y_val, y_pred_lgb)
print(f"\nValidation Accuracy: {lgb_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_lgb))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_lgb))

Training LightGBM Classifier

Validation Accuracy: 0.8975

Classification Report:
              precision    recall  f1-score   support

      user_1       0.88      0.86      0.87       142
      user_2       0.98      0.87      0.93       142
      user_3       0.83      0.97      0.90       116

    accuracy                           0.90       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.90      0.90      0.90       400


Confusion Matrix:
[[122   1  19]
 [ 14 124   4]
 [  2   1 113]]


In [20]:
# XGBoost Classifier
print("=" * 60)
print("Training XGBoost Classifier")
print("=" * 60)

# Encode labels for XGBoost (requires 0, 1, 2 format)
label_encoder_y = LabelEncoder()
y_train_encoded = label_encoder_y.fit_transform(y_train)
y_val_encoded = label_encoder_y.transform(y_val)

xgb_classifier = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    random_state=42,
    eval_metric='mlogloss',
    verbosity=0
)

xgb_classifier.fit(X_train, y_train_encoded)

# Predictions
y_pred_xgb_encoded = xgb_classifier.predict(X_val)
y_pred_xgb = label_encoder_y.inverse_transform(y_pred_xgb_encoded)

# Evaluation
xgb_accuracy = accuracy_score(y_val, y_pred_xgb)
print(f"\nValidation Accuracy: {xgb_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_xgb))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_xgb))

Training XGBoost Classifier

Validation Accuracy: 0.9000

Classification Report:
              precision    recall  f1-score   support

      user_1       0.88      0.86      0.87       142
      user_2       0.99      0.87      0.93       142
      user_3       0.83      0.98      0.90       116

    accuracy                           0.90       400
   macro avg       0.90      0.91      0.90       400
weighted avg       0.91      0.90      0.90       400


Confusion Matrix:
[[122   1  19]
 [ 14 124   4]
 [  2   0 114]]


In [21]:
# CatBoost Classifier
print("=" * 60)
print("Training CatBoost Classifier")
print("=" * 60)

catboost_classifier = CatBoostClassifier(
    iterations=200,
    learning_rate=0.05,
    depth=7,
    random_state=42,
    verbose=0
)

catboost_classifier.fit(X_train, y_train)

# Predictions
y_pred_cat = catboost_classifier.predict(X_val)

# Evaluation
cat_accuracy = accuracy_score(y_val, y_pred_cat)
print(f"\nValidation Accuracy: {cat_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_cat))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_cat))

Training CatBoost Classifier

Validation Accuracy: 0.9050

Classification Report:
              precision    recall  f1-score   support

      user_1       0.90      0.86      0.88       142
      user_2       0.98      0.88      0.93       142
      user_3       0.83      0.99      0.91       116

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400
weighted avg       0.91      0.91      0.91       400


Confusion Matrix:
[[122   1  19]
 [ 13 125   4]
 [  0   1 115]]


In [22]:
# Compare all classifiers
print("=" * 60)
print("Classifier Performance Comparison")
print("=" * 60)
print(f"LightGBM Accuracy:  {lgb_accuracy:.4f}")
print(f"XGBoost Accuracy:   {xgb_accuracy:.4f}")
print(f"CatBoost Accuracy:  {cat_accuracy:.4f}")

# Select the best classifier
classifiers = {
    'LightGBM': (lgb_classifier, lgb_accuracy),
    'XGBoost': (xgb_classifier, xgb_accuracy),
    'CatBoost': (catboost_classifier, cat_accuracy)
}

best_classifier_name = max(classifiers, key=lambda k: classifiers[k][1])
best_classifier = classifiers[best_classifier_name][0]

print(f"\n{'='*60}")
print(f"🏆 Best Classifier: {best_classifier_name}")
print(f"🎯 Best Accuracy: {classifiers[best_classifier_name][1]:.4f}")
print(f"{'='*60}")

Classifier Performance Comparison
LightGBM Accuracy:  0.8975
XGBoost Accuracy:   0.9000
CatBoost Accuracy:  0.9050

🏆 Best Classifier: CatBoost
🎯 Best Accuracy: 0.9050


In [23]:
# Retrain the best classifier on the full training data for final use
print(f"\nRetraining {best_classifier_name} on full training data...")

if best_classifier_name == 'LightGBM':
    final_classifier = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=7,
        num_leaves=31,
        random_state=42,
        verbose=-1
    )
    final_classifier.fit(X_train_encoded, y_train_full)
    
elif best_classifier_name == 'XGBoost':
    final_classifier = xgb.XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=7,
        random_state=42,
        eval_metric='mlogloss',
        verbosity=0
    )
    y_train_full_encoded = label_encoder_y.fit_transform(y_train_full)
    final_classifier.fit(X_train_encoded, y_train_full_encoded)
    
elif best_classifier_name == 'CatBoost':
    final_classifier = CatBoostClassifier(
        iterations=200,
        learning_rate=0.05,
        depth=7,
        random_state=42,
        verbose=0
    )
    final_classifier.fit(X_train_encoded, y_train_full)

print(f"✅ {best_classifier_name} trained on full training data!")
print("\nThis classifier will be used as the Context Detector for the bandit system.")


Retraining CatBoost on full training data...
✅ CatBoost trained on full training data!

This classifier will be used as the Context Detector for the bandit system.


In [33]:
# Create a function to predict user context
def predict_user_context(user_features):
    """
    Predict the user category (user_1, user_2, or user_3) given user features.
    
    Args:
        user_features: DataFrame or array of user features
    
    Returns:
        Predicted user category
    """
    if best_classifier_name == 'XGBoost':
        # XGBoost returns encoded labels, need to inverse transform
        predictions_encoded = final_classifier.predict(user_features)
        return label_encoder_y.inverse_transform(predictions_encoded)
    else:
        return final_classifier.predict(user_features)

# Test the function on a sample
sample_prediction = predict_user_context(X_test_encoded.iloc[:5])
print("Sample predictions on test users:")
print(sample_prediction)

# Display the mapping for reference
print("\n" + "="*60)
print("User Context to Index Mapping (for Bandit):")
print("="*60)
user_to_index = {'user_1': 0, 'user_2': 1, 'user_3': 2}
for user, idx in user_to_index.items():
    print(f"{user} -> Index {idx}")

Sample predictions on test users:
[['user_2']
 ['user_1']
 ['user_1']
 ['user_1']
 ['user_1']]

User Context to Index Mapping (for Bandit):
user_1 -> Index 0
user_2 -> Index 1
user_3 -> Index 2


In [34]:
# MLP (Multi-Layer Perceptron) Neural Network Classifier
import time
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

print("=" * 70)
print("MLP Classifier Training")
print("=" * 70)

# Use the split training data (X_train is 1600 samples from train_test_split of X_train_encoded)
print(f"Using split training data...")
print(f"  - X_train shape: {X_train.shape}, y_train_encoded shape: {y_train_encoded.shape}")
print(f"  - X_val shape: {X_val.shape}, y_val_encoded shape: {y_val_encoded.shape}")

# Create and fit imputer on X_train (matched with y_train_encoded)
imputer = SimpleImputer(strategy='mean')

X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns
)

X_val_imputed = pd.DataFrame(
    imputer.transform(X_val),
    columns=X_val.columns
)

X_test_imputed = pd.DataFrame(
    imputer.transform(X_test_encoded),
    columns=X_test_encoded.columns
)

print(f"✓ Imputed missing values using mean strategy")

# Standardize features for better MLP performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Train MLP classifier
print("\nTraining MLP Classifier...")
start_time = time.time()

mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(128, 64, 32),  # 3 hidden layers with decreasing units
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.2,
    n_iter_no_change=20,
    verbose=0,
    tol=1e-3
)

# Train using the matched training data and labels
mlp_classifier.fit(X_train_scaled, y_train_encoded)
mlp_time = time.time() - start_time

# Make predictions on validation set
y_pred_mlp = mlp_classifier.predict(X_val_scaled)

# Evaluate MLP
mlp_accuracy = accuracy_score(y_val_encoded, y_pred_mlp)

print(f"✓ MLP Training Time: {mlp_time:.4f} seconds")
print(f"✓ MLP Validation Accuracy: {mlp_accuracy:.4f}")

# Store MLP classifier for later use
if 'classifiers' not in locals():
    classifiers = {}
classifiers['MLP'] = mlp_classifier

# Create results summary comparing all classifiers
results_summary = [
    {'Classifier': 'MLP Neural Network', 'Accuracy': mlp_accuracy, 'Training_Time': mlp_time},
    {'Classifier': 'XGBoost', 'Accuracy': xgb_accuracy, 'Training_Time': 0},
    {'Classifier': 'LightGBM', 'Accuracy': lgb_accuracy, 'Training_Time': 0},
    {'Classifier': 'CatBoost', 'Accuracy': cat_accuracy, 'Training_Time': 0}
]

print("\n" + "=" * 70)
print("Classifier Comparison Summary")
print("=" * 70)

results_table = pd.DataFrame(results_summary)
print(results_table.to_string(index=False))

# Find best classifier
best_idx = results_table['Accuracy'].idxmax()
best_classifier_name = results_table.loc[best_idx, 'Classifier']
best_accuracy = results_table.loc[best_idx, 'Accuracy']

print(f"\n✓ Best Classifier: {best_classifier_name} (Accuracy: {best_accuracy:.4f})")

# Update final classifier if MLP is best
if best_classifier_name == 'MLP Neural Network':
    final_classifier = classifiers['MLP']
    print(f"✓ Updated final classifier to MLP Neural Network")
else:
    print(f"✓ MLP score {mlp_accuracy:.4f} is competitive with {best_classifier_name}!")

MLP Classifier Training
Using split training data...
  - X_train shape: (1600, 32), y_train_encoded shape: (1600,)
  - X_val shape: (400, 32), y_val_encoded shape: (400,)
✓ Imputed missing values using mean strategy

Training MLP Classifier...
✓ MLP Training Time: 0.6906 seconds
✓ MLP Validation Accuracy: 0.8325

Classifier Comparison Summary
        Classifier  Accuracy  Training_Time
MLP Neural Network    0.8325       0.690648
           XGBoost    0.9000       0.000000
          LightGBM    0.8975       0.000000
          CatBoost    0.9050       0.000000

✓ Best Classifier: CatBoost (Accuracy: 0.9050)
✓ MLP score 0.8325 is competitive with CatBoost!


# `Contextual Bandit`

## Reward Sampler Initialization

The sampler is initialized using the student's roll number `i`.
Rewards are obtained using `sampler.sample(j)`.


## Arm Mapping

| Arm Index (j) | News Category | User Context |
|--------------|---------------|--------------|
| 0–3          | Entertainment, Education, Tech, Crime | User1 |
| 4–7          | Entertainment, Education, Tech, Crime | User2 |
| 8–11         | Entertainment, Education, Tech, Crime | User3 |

## Epsilon-Greedy Strategy

This section implements the epsilon-greedy contextual bandit algorithm.


## Upper Confidence Bound (UCB)

This section implements the UCB strategy for contextual bandits.

## SoftMax Strategy

This section implements the SoftMax strategy with temperature $ \tau = 1$.


## Reinforcement Learning Simulation

We simulate the bandit algorithms for $T = 10,000$ steps and record rewards.

P.S.: Change $T$ value as and if required.


## Results and Analysis

This section presents:
- Average Reward vs Time
- Hyperparameter comparisons
- Observations and discussion


## Final Observations

- Comparison of Epsilon-Greedy, UCB, and SoftMax
- Effect of hyperparameters
- Strengths and limitations of each approach
